In [ ]:
# --- 1. Інсталяція ---
!pip install diffusers transformers accelerate safetensors
!pip install torch-fidelity torchmetrics[image] lpips scikit-image


# --- 2. Імпорт ---
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
from IPython.display import display

import lpips
from torchmetrics.image.fid import FrechetInceptionDistance
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np
from torchvision import transforms


# --- 3. Ініціалізація SD ---
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe_img2img = pipe_img2img.to("cuda" if torch.cuda.is_available() else "cpu")

# --- 4. Генерація чорновика ---
prompt = "A fantasy castle on a mountain at sunset in Swissaland best village with mounts river."
sd_image = pipe(prompt).images[0]
sd_image.save("sd_output.png")
sd_image.show()
display(sd_image)

# --- 5. Редагування / покращення через img2img ---
# strength 0.6 означає, що зображення буде зберігати деякі деталі оригіналу
edit_prompt = "Make it in a vibrant fantasy painting style with butterflys."
final_image = pipe_img2img(prompt=edit_prompt, image=sd_image, strength=0.6).images[0]
final_image.save("final_output.png")
final_image.show()
display(final_image)



# --- 6.1 Підготовка зображень для LPIPS ---
transform_lpips = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Перетворюємо PIL -> torch.Tensor і нормалізуємо [-1,1] для LPIPS
sd_lpips = transform_lpips(sd_image).unsqueeze(0) * 2 - 1
final_lpips = transform_lpips(final_image).unsqueeze(0) * 2 - 1

# --- 6.2 LPIPS ---
lpips_fn = lpips.LPIPS(net='alex')  # можна також 'vgg'
lpips_score = lpips_fn(sd_lpips, final_lpips)
print("LPIPS між чорновиком і фіналом:", lpips_score.item())

# --- 6.3 PSNR та SSIM ---
# Перетворюємо PIL -> numpy (H,W,C)
# --- Підготовка зображень ---
sd_np = np.array(sd_image.resize((256, 256)))
final_np = np.array(final_image.resize((256, 256)))

# --- PSNR ---
psnr_score = psnr(sd_np, final_np)

# --- SSIM ---
# Тепер замість multichannel=True використовується channel_axis=2 для RGB
ssim_score = ssim(sd_np, final_np, channel_axis=2)

print("PSNR між чорновиком і фіналом:", psnr_score)
print("SSIM між чорновиком і фіналом:", ssim_score)
